### DESCRIPTION:
    This example shows how to create embeddings from Azure Documentation files in PDF format and saves it in the /data/documentation/ folder into a FAISS index.
    Then query these documents and get an answer using OpenAI GPT3.5 with chat
### REQUIREMENTS:
    Create an .env file with your OpenAI API key and save it in the root directory of this project.

  For more information about Faiss index, see:
      https://github.com/facebookresearch/faiss

In [1]:
from langchain.document_loaders import PyPDFLoader 
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

import utils
from dotenv import load_dotenv

In [ ]:
# IMPORTANT!!
# you only need to run this once to create the embeddings, then you can use the faiss_index for retrieval
# you can use the already created faiss_index in the dbs folder - ./dbs/documentation/faiss_index

# create embeddings from Azure Documentation files in PDF format in the /data/documentation/ folder and save it to a FAISS index.
utils.init_OpenAI()
embeddings = utils.init_embeddings()

dataPath = "./data/documentation/"
fileName = dataPath + "azure-azure-functions.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
db = FAISS.from_documents(documents=pages, embedding=embeddings)
db.save_local("./dbs/documentation/faiss_index")

In [3]:

llm = utils.init_llm()
embeddings = utils.init_embeddings()
vectorStore = FAISS.load_local("./dbs/documentation/faiss_index", embeddings)
retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k":2})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

In [10]:
qa({"query": "Which azure functions' version supports Python 3.7?"})

{'query': "Which azure functions' version supports Python 3.7?",
 'result': 'The article does not mention Python 3.7 specifically. It states that the Python v2 programming model is only supported in the 4.x functions runtime. For more information, you can refer to the Azure Functions runtime versions overview.'}

In [11]:
qa({"query": "Explain which application patterns are supported by Azure Functions"})

{'query': 'Explain which application patterns are supported by Azure Functions',
 'result': 'Azure Functions support several application patterns, including:\n\n1. Function chaining: where a sequence of functions executes in a specific order, and the output of one function is applied to the input of another function.\n\n2. Fan-out/fan-in: where multiple functions execute in parallel, and the results are aggregated to produce a final result.\n\n3. Async HTTP APIs: where an HTTP request triggers a long-running process, and the response is returned later when the process is complete.\n\n4. Monitoring: where functions can be monitored for performance, errors, and other metrics using Azure Application Insights.\n\n5. Human interaction: where functions can be used to automate workflows that involve human interaction, such as approval processes.\n\n6. Aggregator (stateful entities): where functions can maintain stateful entities, such as a shopping cart or a user session.'}